## ALGO DEEP DIVE:
Simple Algorithm for Recommendation (SAR) algorithm using a Python single-node implementation.
SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history. It is powered by understanding the similarity between items, and recommending similar items to those a user has an existing affinity for.

<br/>At a very high level, two intermediate matrices are created and used to generate a set of recommendation scores:
<br/>A)An item similarity matrix  estimates item-item relationships.
<br/>B)An affinity matrix  estimates user-item relationships.
<br/>C)Recommendation scores are then created by computing the matrix multiplication.

<br/>**Advantages of SAR:**
<br/>-High accuracy for an easy to train and deploy algorithm
<br/>-Fast training, only requiring simple counting to construct matrices used at prediction time.
<br/>-Fast scoring, only involving multiplication of the similarity matrix with an affinity vector
<br/>**Notes to use SAR properly:**
<br/>Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
<br/>It's memory-hungry, requiring the creation of an  sparse square matrix (where  is the number of items). This can also be a problem for many matrix factorization algorithms.


<br/>**1)Compute item co-occurrence and item similarity**
<br/>SAR defines similarity based on item-to-item co-occurrence data. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a mXm matrix C, where c_ij is the number of times item i occurred with item j, and m is the total number of items.

<br/>Once we have a co-occurrence matrix, an item similarity matrix  can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include Jaccard, lift, and counts (meaning no rescaling).

<br/>If c_ii and c_jj are the ith and jth diagonal elements of C, the rescaling options are:
<br/>**-Jaccard**: s_ij = c_ij/ (c_ii + c_jj - c_ij)
<br/>**-lift**: s_ij = c_ij/ (c_ii * c_jj)
<br/>**-counts**: s_ij = c_ij

In general, using counts as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. lift by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. Jaccard is a compromise between the two.

<br/>**2)Compute user affinity scores**
<br/>The affinity matrix in SAR captures the strength of the relationship between each individual user and the items that user has already interacted with. SAR incorporates two factors that can impact users' affinities:

<br/>-It can consider information about the ***type*** of user-item interaction through differential weighting of different events (e.g. it may weigh events in which a user rated a particular item more heavily than events in which a user viewed the item).
<br/>-It can consider information about ***when*** a user-item event occurred (e.g. it may discount the value of events that take place in the distant past.

Formula for getting affinity score for each user-item pair is-
<br/>***a_{ij}=sum_k w_k * (0.5)^{{t_0-t_k}/{T}}***
<br/>where a_ij is the affinity score, w_k is the interaction weight,  t_0 is a reference time, t_k is the timestamp for the k-th interaction, and T is a hyperparameter that controls the speed of

<br/>**3)Remove seen item***
<br/>Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

<br/>**4)Top-k item calculation***
The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix (A) by the similarity matrix (S). The result is a recommendation score matrix, where each row corresponds to a user, each column corresponds to an item, and each entry corresponds to a user / item pair. Higher scores correspond to more strongly recommended items.

In [0]:
%pip install recommenders
%pip install datetime

Python interpreter will be restarted.
Requirement already satisfied: recommenders in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (1.1.0)
Requirement already satisfied: memory-profiler<1,>=0.54.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (0.60.0)
Requirement already satisfied: cornac<2,>=1.1.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.14.2)
Requirement already satisfied: transformers<5,>=2.5.0 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (4.15.0)
Requirement already satisfied: tqdm<5,>=4.31.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (4.59.0)
Requirement already satisfied: seaborn<1,>=0.8.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (0.11.1)
Requirement already satisfied: jinja2<3.1,>=2 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (2.11.2)
Requirement already satisfied: category-encoders<2,>=1.3.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.3.0)
Requirement already satisfied: pyyaml<6,>=5.4.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (5.4.1)
Requirement already satisfied: pandas<2,>1.0.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.4.2)
Requirement already satisfied: bottleneck<2,>=1.2.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.3.4)
Requirement already satisfied: requests<3,>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (2.25.1)
Requirement already satisfied: matplotlib<4,>=2.2.2 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (3.4.2)
Requirement already satisfied: nltk<4,>=3.4 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (3.6.1)
Requirement already satisfied: scipy<2,>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (1.5.2)
Requirement already satisfied: scikit-surprise>=1.0.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.1.1)
Requirement already satisfied: lightgbm>=2.2.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (3.2.1)
Requirement already satisfied: retrying>=1.3.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.3.3)
Requirement already satisfied: numba<1,>=0.38.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (0.53.1)
Requirement already satisfied: lightfm<2,>=1.15 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (1.16)
Requirement already satisfied: pandera[strategies]>=0.6.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69fba55c-12a3-4874-8dff-8ef66f2999be/lib/python3.8/site-packages (from recommenders) (0.10.1)
Requirement already satisfied: numpy>=1.19 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (1.19.5)
Requirement already satisfied: scikit-learn<1.0.3,>=0.22.1 in /databricks/python3/lib/python3.8/site-packages (from recommenders) (0.22.2.post1)
Requirement already satisfied: statsmodels>=0.6.1 in /databricks/python3/lib/python3.8/site-packages (from category-encoders<2,>=1.3.0->recommenders) (0.11.1)
Requirement already satisfied: patsy>=0.4.1 in /databricks/python3/lib/python3.8/site-packages (from category-encoders<2,>=1.3.0->recommenders) (0.5.1)
Requirement already satisfied: powerlaw in /local_disk0/.ephemeral_nfs/envs/pythonEnv-69f

In [0]:
# set the environment path to find Recommenders
import logging
import numpy as np
import pandas as pd
#import scrapbook as sb
from sklearn.preprocessing import minmax_scale

from recommenders.utils.python_utils import binarize
from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)
from recommenders.models.sar import SAR
import sys

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

/databricks/python/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/databricks/python/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/databricks/python/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
System version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
Pandas version: 1.4.2

In [0]:
df1 = spark.read.format("csv").options(header='true', delimiter = ',').load(".../mldata/MoviesDataRecommendation/ratings.csv")
df_ratings = df1.toPandas()

#Datatype conversion
df_ratings.userId = df_ratings['userId'].astype('int')
df_ratings.movieId = df_ratings['movieId'].astype('int')
df_ratings.rating = df_ratings['rating'].astype('float')
df_ratings.timestamp = df_ratings['timestamp'].astype('int')
print(df_ratings.dtypes)



# top k items to recommend
TOP_K = 10 
COL_USER = "userId"
COL_ITEM = "movieId"
COL_RATING = "rating"
COL_PREDICTION = "rating"
COL_TIMESTAMP = "timestamp"

userId int64
movieId int64
rating float64
timestamp int64
dtype: object

In [0]:
df_ratings = df_ratings.head(100000)
#Memory intensive so lower the count

In [0]:
''' Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided python_stratified_split function which holds out a percentage (in this case 30%) of items from each user, but ensures all users are in both train and test datasets.
'''
df=df_ratings.copy()
train, test = python_stratified_split(
    df, filter_by="user", min_rating=10, ratio=0.7,
    col_user=COL_USER, col_item=COL_ITEM
)
print("Size of Train: ",train.shape[0])
print("Size of Test: ",test.shape[0])

Size of Train: 69992
Size of Test: 30008

In [0]:
#Modelling
'''
Parameters:
-col_user (str): user column name
-col_item (str): item column name
-col_rating (str): rating column name
-col_timestamp (str): timestamp column name
-col_prediction (str): prediction column name
-similarity_type (str): ['cooccurrence', 'jaccard', 'lift'] option for computing item-item similarity
-time_decay_coefficient (float): number of days till ratings are decayed by 1/2
-time_now (int | None): current time for time decay calculation
-timedecay_formula (bool): flag to apply time decay
-threshold (int): item-item co-occurrences below this threshold will be removed
-normalize (bool): option for normalizing predictions to scale of original ratings
'''
#Training
header = {
    "col_user": "userId",
    "col_item": "movieId",
    "col_rating": "rating",
    "col_timestamp": "timestamp",
    "col_prediction": "prediction",
}
model = SAR(
    col_user="userId",
    col_item="movieId",
    col_rating="rating",
    col_timestamp="timestamp",
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    timedecay_formula=True,
    normalize=True
)
model.fit(train)

#Scoring, remove already rated/seen movie or item
top_k = model.recommend_k_items(test, remove_seen=True) 
top_k['prediction']= round(top_k['prediction'],2)
print(top_k[top_k['prediction']>0])

userId movieId prediction
0 1 102666 0.92
1 1 146028 0.92
2 1 107412 0.92
3 1 26184 0.92
4 1 172077 0.92
... ... ... ...
7565 757 1387 2.96
7566 757 1206 2.96
7567 757 1265 2.95
7568 757 1222 2.91
7569 757 2406 2.86

[7570 rows x 3 columns]

In [0]:
# all ranking metrics have the same arguments
eval_map = map_at_k(test, top_k, col_user='userId', col_item='movieId', col_rating='rating', k=TOP_K)
eval_ndcg = ndcg_at_k(test, top_k, col_user='userId', col_item='movieId', col_rating='rating', k=TOP_K)
eval_precision = precision_at_k(test, top_k, col_user='userId', col_item='movieId', col_rating='rating', k=TOP_K)
eval_recall = recall_at_k(test, top_k, col_user='userId', col_item='movieId', col_rating='rating', k=TOP_K)
eval_rmse = rmse(test, top_k, col_user='userId', col_item='movieId', col_rating='rating')
eval_mae = mae(test, top_k, col_user='userId', col_item='movieId', col_rating='rating')
eval_rsquared = rsquared(test, top_k, col_user='userId', col_item='movieId', col_rating='rating')
eval_exp_var = exp_var(test, top_k, col_user='userId', col_item='movieId', col_rating='rating')

print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision:\t {eval_precision:f}",
      f"Recall:\t {eval_recall:f}", 
      f"RMSE:\t {eval_rmse:f}", 
      f"rsquared:\t {eval_rsquared:f}", 
      f"exp_variance:\t {eval_exp_var:f}",sep='\n')

Model:
Top K: 10
MAP: 0.071858
NDCG: 0.253356
Precision: 0.229062
Recall: 0.117728
RMSE: 1.492350
rsquared: -1.530726
exp_variance: -0.129740

In [0]:
#prediction v/s original rating in test dataframe
top_k.merge(test, on=['userId','movieId'], how='inner')


Out[23]:

,userId,movieId,prediction,rating,timestamp
0,2,1198,3.08,4.0,1141417032
1,2,1580,3.05,4.5,1141417059
2,2,2028,2.98,4.0,1141415840
3,3,69526,1.86,4.0,1439473894
4,4,94864,2.59,0.5,1573939936
...,...,...,...,...,...
1729,757,1917,2.99,1.0,1184013974
1730,757,1387,2.96,4.0,1184014485
1731,757,1206,2.96,4.0,1184015591
1732,757,1265,2.95,4.0,1184014201
